In [2]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

import ccxt

from tp_utils.data_provider import read_prices, read_data, market_to_symbol, load_data_from_exchange
from Portfolio.portfolio_tools import *
from tp_utils.tp_utils import  create_market_buy_order,  create_market_sell_order, get_balance
from tp_config import *

pd.set_option("display.max_rows", 90)

### Установить подключение

In [3]:
from pybit.unified_trading import HTTP

main_api = "5ERAemyDtATcprZyaA"
main_secret = "KKw2m9BudPFonS6Y5nfsGqKRHTiwzSSoikDv"


session = HTTP(
    api_key = main_api,
    api_secret = main_secret,
    recv_window = 10000
)

subaccounts = { 
    "RiskfolioMaxRet" : 162098887,
    "Semicovariance"  : 158552201
}

In [5]:
from bybit import  *

In [29]:
def get_all_assets_main(session : HTTP):
    """
    Получаю остатки на аккаунте по всем монетам
    :param cl:
    :return: dict
    """
    r = session.get_wallet_balance(accountType="UNIFIED")
    assets = {
        asset.get('coin') : float(asset.get('availableToWithdraw', '0.0'))
        for asset in r.get('result', {}).get('list', [])[0].get('coin', [])
    }
    return assets

def get_all_assets_sub(session, subaccount : str):
    
    memberId = subaccounts[subaccount]
    res = session.get_coins_balance(
        memberId = memberId,
        accountType="UNIFIED")
    
    assets = {asset['coin']:asset['walletBalance']
              for asset in res['result']['balance'] if float(asset['walletBalance'])}
    return assets

def get_all_assets(session, subaccount = None):
    if subaccount is None:
        assets = get_all_assets_main(session)  
    else:
        assets = get_all_assets_sub(session, subaccount)
        
    return assets
        
def get_asset(cl : HTTP, coin, subaccount = None):
    """
    Получаю остатки на аккаунте по конкретной монете
    :param cl:
    :param coin:
    :return:
    """
    assets = get_all_assets(cl, subaccount)
    return float(assets.get(coin, 0.0))

def get_subaccounts_id():
    res = session.get_sub_uid()
    return res['result']['subMemberIds']


In [60]:
def rd(val, dec):
    factor = 1 / (10 ** dec)
    return (val // factor) * factor




In [22]:
res = get_all_assets(session, "RiskfolioMaxRet")
print(res)



{'TOKEN': '0.00895', 'AVAX': '0.000893', 'TON': '0.00766', 'FLT': '0.00266', 'SOL': '0.000912', 'ICP': '0.01676', 'BNB': '0.00000656', 'USDT': '101.0806'}


In [30]:
get_asset(session,  "USDT", "RiskfolioMaxRet")

101.0806

## Balance

In [54]:
df = pd.DataFrame(res.items(), columns = ['Token', 'Amount'])
df = df[df['Amount'] > 0.1]
df

,Token,Amount
2,TON,119.90350000
4,MNT,0.86798147
11,PEPE,0.16900000
12,LUNA,49.44286764


## Market buy

In [37]:
price_buy = create_market_buy_order(exchange, market, amount)
print('price', price_buy, ', sum', price_buy * amount)

price 0.071306 , sum 0.0142612


## Market Sell

In [15]:
price_sell = create_market_sell_order(exchange, market, amount)
print(price_sell, 'sum', price_sell * amount)

0.00018281 sum 0.018281


### Limit sell

In [21]:
order_type = 'stop_loss_limit'  # or 'market', other types aren't unified yet
side = 'sell'
stop_price = 0.0002150
sell_price = 0.0002140
# your price
# overrides
params = {
    'stopPrice': stop_price,  # your stop price
}
order = exchange.create_order(symbol, order_type, side, amount, sell_price, params)

In [51]:
exchange.cancel_all_orders(symbol)

[{'info': {'symbol': 'DOGEBTC',
   'origClientOrderId': 'web_654918598cbd4910a3cb74fe677b92bf',
   'orderId': '239881844',
   'orderListId': '-1',
   'clientOrderId': 'SqiOk3B2fx3pQKDBISxZmL',
   'price': '0.00000578',
   'origQty': '553.00000000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'CANCELED',
   'timeInForce': 'GTC',
   'type': 'STOP_LOSS_LIMIT',
   'side': 'SELL',
   'stopPrice': '0.00000579'},
  'id': '239881844',
  'clientOrderId': 'SqiOk3B2fx3pQKDBISxZmL',
  'timestamp': None,
  'datetime': None,
  'lastTradeTimestamp': None,
  'symbol': 'DOGE/BTC',
  'type': 'stop_loss_limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'sell',
  'price': 5.78e-06,
  'stopPrice': 5.79e-06,
  'amount': 553.0,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 553.0,
  'status': 'canceled',
  'fee': None,
  'trades': [],
  'fees': []}]

# Sell All

In [56]:
df = get_balance(exchange)
df = df[(df['free'].astype(float) +df['locked'].astype(float)) > 0]
df = df[~df['asset'].isin(delisted_assets)]
for row in df.iterrows():
    asset = row[1]['asset']
    amount = row[1]['free']
    locked = row[1]['locked']
    if asset in ['BTC', 'USDT', 'BNB']:
        continue

    market = asset + '-BTC' 
    ticker = asset + '/BTC'
    if float(locked) > 0:
        exchange.cancel_all_orders(ticker)
        
    try:
        price_sell = create_market_sell_order(exchange, market, amount)
        print(market, price_sell)
    except:
        pass

ETH-BTC 0.069013
QTUM-BTC 0.0002347
DATA-BTC 2.81e-06
THETA-BTC 0.00015202
VET-BTC 2.34e-06
DOGE-BTC 5.64e-06
FTT-BTC 0.0010006
DREP-BTC 1.464e-05
SOL-BTC 0.00085663
DOT-BTC 0.00045237
UNI-BTC 0.0006086
PSG-BTC 0.0009337
TVK-BTC 7.17e-06


### Trades

In [988]:
raw_trades =  exchange.fetch_my_trades('TFUEL/BTC')
trades = [[round(x['timestamp']/1000), x['side'], x['amount'], x['price'], x['cost']] for x in raw_trades]

In [ ]:
df  = pd.DataFrame(trades, columns = ['time', 'side', 'amount', 'price', 'cost'])
dfx = df.groupby(['time', 'side']).sum()
dfx['price'] = dfx['cost']/dfx['amount']
dfx.sort_values('time', ascending = False)[:20]  

In [ ]:
portfolios = ['max_sharpe', 'main', 'index_low', 'efficient_risk']
for portfolio in portfolios:
    portfolio_file = portfolio + '.json'

    with open(portfolio_file) as f:
        portfolio_data = json.load(f)
    portfolio = portfolio_data['markets']
    sell_portfolio(exchange, portfolio)

In [327]:
d = {"stop":not True}
with open("stop.json", 'w') as f:
    json.dump(d, f)

In [328]:
with open("stop.json") as f:
    x = json.load(f)
x

{'stop': False}

## Portfolio recalc

### 0. Data from exchange

In [43]:
pair = 'USDT'
index_assets = pd.read_csv('Portfolio/index_assets.csv')['asset'].tolist()
markets_index = [s + '-' + pair for s in index_assets]

index_low_assets = pd.read_csv('Portfolio/index_low_assets.csv')['asset'].tolist()
index_all = index_assets + index_low_assets
markets = [s + '-' + pair for s in index_all]

In [20]:
exchange = ccxt.bybit()
pair = 'USDT'
assets = ['ADA', 'APT', 'AVAX', 'BNB', 'SOL', 'TRX', 'ZIL', 'NEON',  'WLD', 'PEPE', 'NEAR', 'ETH', 'BONK', 'WAVES',
          'FTT',  'FTM', 'MATIC', 'INJ', 'JUP', 'SEI', 'SUI', 'THETA', 'RUNE', 'GALA', 'PYTH', 'PPT', 'FIRE', 'NGL',
         'CYBER']

assets.sort()

#markets = [m + '-' + pair for m in assets]

start_timestamp = exchange.parse8601('2023-04-01 00:00:00')
timeframes = {'1h':60, '4h':60}
data_path = DATA_PATH_CRYPTO + '/' + pair
print(data_path)
load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path, 10, verbose = False)

D:\Data\Crypto\ByBit/USDT


D:\Code\Projects\QuantMod\tp\tp_utils\data_provider.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])
D:\Code\Projects\QuantMod\tp\tp_utils\data_provider.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_old, df_new])
D:\Code\Projects\QuantMod\tp\tp_utils\data_provider.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-N

Done
Time taken = 0.045  hours


### 1. Set index assets

In [44]:
markets = [x for x in markets if x != 'FLT-USDT']

### 2. Load data

In [45]:
df_prices =  load_data_for_portfolio(markets, '4h')   
print(datetime.fromtimestamp(df_prices.index[-1]/1000))

AVAX-USDT
BTC-USDT
BNB-USDT
DOT-USDT
DOGE-USDT
ETH-USDT
LINK-USDT
LTC-USDT
LUNA-USDT
MATIC-USDT
MNT-USDT
SOL-USDT
TON-USDT
TRX-USDT
AAVE-USDT
ADA-USDT
AEVO-USDT
ALGO-USDT
APT-USDT
AXS-USDT
BONK-USDT
BOME-USDT
CAKE-USDT
CYBER-USDT
CPOOL-USDT
EOS-USDT
ETC-USDT
ETHFI-USDT
FIRE-USDT
FTT-USDT
FTM-USDT
GALA-USDT
INJ-USDT
ICP-USDT
JUP-USDT
NEON-USDT
NEAR-USDT
ONDO-USDT
PEPE-USDT
PENDLE-USDT
PPT-USDT
PYTH-USDT
RUNE-USDT
SEI-USDT
SQT-USDT
SUI-USDT
THETA-USDT
TOKEN-USDT
UNI-USDT
WAVES-USDT
WLD-USDT
XRP-USDT
ZIL-USDT
2024-03-28 08:00:00


In [50]:
df_prices

,AVAX-USDT,BTC-USDT,BNB-USDT,DOT-USDT,DOGE-USDT,ETH-USDT,LINK-USDT,LTC-USDT,LUNA-USDT,MATIC-USDT,...,SEI-USDT,SQT-USDT,SUI-USDT,THETA-USDT,TOKEN-USDT,UNI-USDT,WAVES-USDT,WLD-USDT,XRP-USDT,ZIL-USDT
T,,,,,,,,,,,,,,,,,,,,,
1696118400000,9.3307,27048.82,215.2861,4.116,0.06207,1674.39,8.1613,65.97,0.4835,0.5339,...,0.1207,NaN,0.4709,0.6594,NaN,4.4600,1.5826,1.6986,0.5168,0.01760
1696132800000,9.4524,27111.40,214.9711,4.139,0.06216,1678.48,8.0564,66.18,0.4879,0.5361,...,0.1218,NaN,0.4754,0.6548,NaN,4.5078,1.5801,1.6783,0.5178,0.01756
1696147200000,9.6361,27194.02,216.1613,4.201,0.06273,1688.83,8.1226,67.41,0.4890,0.5535,...,0.1220,NaN,0.4813,0.6627,NaN,4.5738,1.5848,1.7131,0.5210,0.01784
1696161600000,9.6086,27154.01,215.5800,4.190,0.06252,1683.72,8.0128,66.88,0.4861,0.5559,...,0.1229,NaN,0.4802,0.6614,NaN,4.5628,1.5947,1.7140,0.5190,0.01773
1696176000000,9.5027,27105.96,214.1373,4.196,0.06198,1676.20,7.8425,66.47,0.4816,0.5539,...,0.1227,NaN,0.4890,0.6599,NaN,4.5308,1.5902,1.6915,0.5162,0.01755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711540800000,53.3232,68852.02,569.9740,9.379,0.18766,3513.81,19.1452,93.65,1.1000,1.0219,...,0.8554,0.03162,2.1055,2.9331,0.20755,12.3676,3.7982,8.5908,0.6109,0.03510
1711555200000,53.8515,68642.87,569.4129,9.413,0.18274,3491.44,19.2696,93.82,1.1021,1.0090,...,0.8674,0.02990,2.1379,2.9580,0.21202,12.2070,3.7934,8.4421,0.6143,0.03528
1711569600000,54.1103,69470.79,574.1989,9.442,0.19055,3502.29,19.2812,93.55,1.0984,1.0089,...,0.8931,0.03513,2.0750,2.9809,0.20191,12.2348,3.7922,8.4225,0.6119,0.03546


### 3. Calc weights

#### 3.1 Calc weights 

In [54]:
lookback = 120
df_period = df_prices.iloc[-lookback:]
df_period = df_period.dropna(axis = 1)
print_data(df_period)

ef = calc_frontier(df_period, "ledoit_wolf",  "ema_historical_return", span = 150)
#ef = calc_frontier(df_period, 'semicovariance', "ema_historical_return")


#opt_method = "efficient_risk"
opt_method = 'max_sharpe'

if opt_method == 'max_sharpe': 
    dfw = calc_weights(ef, opt_method, 0)
else:
    dfw = calc_weights(ef, opt_method, 0.25)
print(opt_method)    
dfw 

2024-03-08 12:00:00
2024-03-28 08:00:00
max_sharpe


,weights
CPOOL-USDT,0.29556
SQT-USDT,0.32867
TOKEN-USDT,0.37577


In [53]:
df_prices.iloc[-lookback:]

,AVAX-USDT,BTC-USDT,BNB-USDT,DOT-USDT,DOGE-USDT,ETH-USDT,LINK-USDT,LTC-USDT,LUNA-USDT,MATIC-USDT,...,SEI-USDT,SQT-USDT,SUI-USDT,THETA-USDT,TOKEN-USDT,UNI-USDT,WAVES-USDT,WLD-USDT,XRP-USDT,ZIL-USDT
T,,,,,,,,,,,,,,,,,,,,,
1709884800000,43.5468,67743.98,474.0608,10.431,0.16890,3950.80,20.0004,87.90,1.1422,1.1584,...,0.9085,0.01515,1.5463,3.0511,0.08746,15.0458,3.3298,7.2805,0.6284,0.03458
1709899200000,42.3574,67614.50,472.7876,10.217,0.16345,3900.41,19.6896,86.46,1.0977,1.1177,...,0.8783,0.01497,1.5080,2.9992,0.09057,14.5152,3.2090,7.1855,0.6142,0.03347
1709913600000,42.5434,68869.12,484.4892,10.490,0.16768,3928.74,19.7510,87.01,1.1491,1.1281,...,0.8959,0.01566,1.5239,2.9761,0.08856,14.6107,3.3036,7.5098,0.6199,0.03410
1709928000000,42.6165,68108.02,485.7265,10.635,0.16454,3883.20,19.6928,88.34,1.1393,1.1247,...,0.9250,0.01515,1.5430,3.0452,0.08729,14.5991,3.3438,7.5272,0.6199,0.03489
1709942400000,42.5875,68232.70,489.1490,10.640,0.16908,3922.81,20.0458,88.83,1.1555,1.1371,...,0.9066,0.01501,1.5346,3.1349,0.09179,15.0036,3.3605,9.1280,0.6256,0.03534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711540800000,53.3232,68852.02,569.9740,9.379,0.18766,3513.81,19.1452,93.65,1.1000,1.0219,...,0.8554,0.03162,2.1055,2.9331,0.20755,12.3676,3.7982,8.5908,0.6109,0.03510
1711555200000,53.8515,68642.87,569.4129,9.413,0.18274,3491.44,19.2696,93.82,1.1021,1.0090,...,0.8674,0.02990,2.1379,2.9580,0.21202,12.2070,3.7934,8.4421,0.6143,0.03528
1711569600000,54.1103,69470.79,574.1989,9.442,0.19055,3502.29,19.2812,93.55,1.0984,1.0089,...,0.8931,0.03513,2.0750,2.9809,0.20191,12.2348,3.7922,8.4225,0.6119,0.03546


In [39]:
df_period

,AVAX-USDT,BTC-USDT,BNB-USDT,DOT-USDT,ETH-USDT,LINK-USDT,LTC-USDT,LUNA-USDT,MATIC-USDT,SOL-USDT_x,...,SOL-USDT_y,SQT-USDT,SUI-USDT,THETA-USDT,TOKEN-USDT,UNI-USDT,WAVES-USDT,WLD-USDT,XRP-USDT,ZIL-USDT
T,,,,,,,,,,,,,,,,,,,,,
1709798400000,42.8290,66903.20,438.6086,10.457,3787.80,19.9208,86.20,1.0911,1.1327,140.29,...,140.29,0.01445,1.5332,2.9109,0.07128,14.8438,3.4269,7.2034,0.6220,0.03522
1709812800000,43.7806,67254.54,449.1878,10.427,3837.74,20.0827,86.60,1.1619,1.1578,145.93,...,145.93,0.01482,1.5370,2.9223,0.07107,15.1044,3.3408,7.1848,0.6252,0.03501
1709827200000,43.8365,67497.25,467.9742,10.435,3870.42,20.1023,88.79,1.1364,1.1688,148.78,...,148.78,0.01568,1.5248,2.9182,0.07637,14.8004,3.4031,7.1361,0.6325,0.03521
1709841600000,43.1978,66818.02,474.5266,10.404,3868.64,20.0843,87.97,1.1558,1.1605,143.83,...,143.83,0.01518,1.5805,2.9530,0.07288,14.8879,3.3698,7.1811,0.6270,0.03539
1709856000000,42.9408,67150.48,464.7146,10.409,3901.00,19.9861,88.73,1.1747,1.1573,144.31,...,144.31,0.01572,1.5564,3.2683,0.07581,15.4050,3.3488,7.2799,0.6283,0.03549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711454400000,56.6867,70244.69,579.6548,9.803,3593.29,20.1858,88.93,1.2022,1.0454,190.35,...,190.35,0.02562,1.8188,3.1044,0.22718,12.5623,3.9813,8.8604,0.6342,0.03683
1711468800000,55.6192,69472.15,575.4255,9.555,3560.84,19.6603,95.10,1.1783,1.0337,188.32,...,188.32,0.03128,1.8625,3.0526,0.22446,12.5459,3.9752,8.6132,0.6297,0.03632
1711483200000,55.7790,69997.04,580.3742,9.689,3587.24,20.0098,95.76,1.1804,1.0458,190.43,...,190.43,0.03366,1.8851,3.0617,0.21983,12.6937,4.0349,8.7439,0.6319,0.03735


### 3.2 Calc weghts min. cardinality

In [24]:
import riskfolio as rp

rms= ['MV', 'MSV',  'CVaR']

def riskfolio_weights(df_period, rm , obj):
    """
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    port.solvers = ['MOSEK']
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 2
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

print_data(df_period)
df_period = df_period.dropna(axis = 1)

for rm in rms:
    print(rm)
    w = riskfolio_weights(df_period, rm, 'MaxRet')
    print(w)

2024-03-08 12:00:00
2024-03-28 08:00:00
MV
You must convert self.cov to a positive definite matrix
                weights
CPOOL-USDT  0.550616484
SQT-USDT    0.443608050
MSV
You must convert self.cov to a positive definite matrix
                weights
CPOOL-USDT  0.550616484
SQT-USDT    0.443608050
CVaR
You must convert self.cov to a positive definite matrix
                weights
CPOOL-USDT  0.550616484
SQT-USDT    0.443608050


In [25]:
# First we need to delete the cardinality constraint
port.card = None 

# Then we need to set the constraint on the minimum number of effective assets
port.nea = 2

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w[w.weights > 0.01])

,weights
ETHFI-USDT,0.016718167
ONDO-USDT,0.462106652
TOKEN-USDT,0.521174797


In [28]:
# Number of assets in portfolio
n_assets = np.sum(np.where(np.round(w,4) > 0, 1, 0)).item()

# Number of effective assets in portfolio
nea = 1/np.sum(w ** 2).item()

print('Number of Assets:', n_assets)
print('Number of Effective Assets:', nea)

Number of Assets: 14
Number of Effective Assets: 9.99999858246235


D:\Soft\Anaconda\envs\t\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


### 4. Sell portfolio

In [18]:
portfolio = portfolio_data['markets']
balance = sell_portfolio(portfolio)
print('Current balance:', balance)  

{'opt_method': 'max_sharpe', 'base': ['index'], 'markets': {'ADA-BTC': 647.583133356188, 'AVAX-BTC': 19.2435189120272, 'DOGE-BTC': 1595.947219604147, 'ETC-BTC': 17.649557037172347, 'ONT-BTC': 105.2389705882353, 'RUNE-BTC': 8.051865907653385, 'XMR-BTC': 0.5688474256022674}}
Current balance: 0.09769360876527733


In [39]:
balance = 0.2

### 5. Set new portfolio

In [40]:
portfolio = set_new_portfolio(exchange,dfw, balance)
portfolio

{'ADA-BTC': 1220.7858048162232,
 'CTSI-BTC': 1544.097807757167,
 'ETC-BTC': 33.56746946583607,
 'ETH-BTC': 0.34019285042333025,
 'ONT-BTC': 491.70230966638155,
 'RUNE-BTC': 15.038899247733559,
 'LTC-BTC': 0.41952,
 'XVG-BTC': 2419.512195121951}

### 6. Buy portfolio

In [41]:
for market in portfolio.keys():
    amount = portfolio[market]
    print(market, amount)
    price_buy =  create_market_buy_order(exchange, market, amount)

portfolio_data['markets'] = portfolio    
with open(portfolio_file, 'w') as f:
    json.dump(portfolio_data, f)    

ADA-BTC 1220.7858048162232
CTSI-BTC 1544.097807757167
ETC-BTC 33.56746946583607
ETH-BTC 0.34019285042333025
ONT-BTC 491.70230966638155
RUNE-BTC 15.038899247733559
LTC-BTC 0.41952
XVG-BTC 2419.512195121951


### End

## Stop limit orders

In [27]:
side = 'sell'
stop_price = 6
limit_price = 6920
params = {'stopPrice': stop_price }
order =  exchange.createOrder(symbol, 'STOP_LOSS_LIMIT', side, amount, limit_price, params)
print(order['id'])

1714800797


In [ ]:
inf = exchange.cancel_order(order['id'], symbol)

In [ ]:
#Типы поддерживаемых ордеров

from pprint import pprint
import ccxt
exchange = ccxt.binance()
exchange.load_markets()
pprint(exchange.markets['BTC/USDT'])

## Skew

In [3]:
from scipy.stats import skew, skewtest

In [1]:
timeframes = ['1m', '3m', '5m',  '15m', '30m', '1h', '2h']

In [20]:
skew_list = []
for tf in timeframes:
    prices = read_data('BTC-USDT', tf)
    dret = daily_returns(prices['C'])
    skew_list.append([tf, skew(dret)])
    
df_skew = pd.DataFrame(skew_list)    
df_skew.columns = ['tf', 'skew']    

In [21]:
skew_treshold = 0.15
df_skew[df_skew['skew'] > skew_treshold]

,tf,skew
0,1m,0.294967222
1,3m,0.675015277
2,5m,0.355812883
4,30m,0.404607105
5,1h,0.207761672
7,4h,0.834205604
8,6h,0.556794138


In [126]:
round(10.23234234, 2)

10.23